### Featurized Linear Regression
Given labeled training data $D_{train}=\{(x_i, y_i)\}$, where $x_i \in \mathbb R^d$, $y_i \in \mathbb R$, and $n \gg d$. This problem is about featurized linear regression, i.e. we can predict $y_i$ as a linear function of a set of feature transformations $\phi(x_i)$. Additionally, we are required to use the following features to make predictions: 
- Linear: $\phi_1(x)=x_1, \space \phi_2(x)=x_2, \space \phi_3(x)=x_3, \space \phi_4(x)=x_4, \space, \phi_5(x)=x_5$
- Quadratic: $\phi_6(x)=x_1^2, \space \phi_7(x)=x_2^2, \space \phi_8(x)=x_3^2, \space \phi_9(x)=x_4^2, \space \phi_{10}(x)=x_5^2$
- Exponential: $\phi_{11}(x)=e^{x_1}, \space \phi_{12}(x)=e^{x_2}, \space \phi_{13}(x)=e^{x_3}, \space \phi_{14}(x)=e^{x_4}, \space \phi_{15}(x)=e^{x_5}$
- Cosine: $\phi_{16}(x)=\cos(x_1), \space \phi_{17}(x)=\cos(x_2), \space \phi_{18}(x)=\cos(x_3), \space \phi_{19}(x)=\cos(x_4), \space \phi_{20}(x)=\cos(x_5)$
- Constant: $\phi_{21}(x)=1$

Now, define $\phi(x):=[\phi_1(x), ..., \phi_{21}(x)]^T$. Then, for a given $x \in \mathbb R^5$, we make the prediction $y=\sum_{i=1}^{21}w_i\phi_i(x)=w^T\phi(x)$, where we have to "learn" the optimal $w$. 

 #### Imports & Reading Input 

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error

data = pd.read_csv("train.csv")
y = data["y"].to_numpy()
data = data.drop(columns=["Id", "y"])
X = data.to_numpy()
n, d, p = 700, 5, 21

#### Transform data

Above we have loaded our $x_i$'s into the matrix: $$X=\begin{bmatrix} - & x_1 & -  \\  & \vdots &  \\ - & x_n & - \end{bmatrix}\in \mathbb R^{n,5}$$ Now, we have to transform each $x_i$ to $\phi(x_i)$, which yields the transformation matrix: $$\Phi = \begin{bmatrix} - & \phi(x_1) & - \\ & \vdots & \\  - & \phi(x_n) & - \end{bmatrix} \in \mathbb R^{n,21}$$

In [2]:
Phi = np.zeros((700, 21))

Phi[:, 0:5] = X[:, 0:5]
Phi[:, 5:10] = X[:, 0:5] ** 2
Phi[:, 10:15] = np.exp(X[:, 0:5])
Phi[:, 15:20] = np.cos(X[:, 0:5])
Phi[:, 20] = 1

#### Fit data
Now, we have to solve the following problem: $$\hat w = \arg \min_{w \in \mathbb R^{21}} \frac 1 {2n} \| y - \Phi w \|_2^2$$The gradient of the loss function $L(w)=\|y - \Phi w \|_2^2$ is $\triangledown_w L(w) = \frac 1 n (\Phi^T \Phi w - \Phi^T y)$. Note that this problem provides a closed-form solution, however, we implement gradient descent with momentum manually. Note that the loss function is convex, thus GD will converge to a global minimum. 

In [3]:
alpha, beta = 10E-2, 0.90
w_old, w_curr = np.zeros(p), np.zeros(p)

for i in range(100): 
    grad = (1 / n) * (Phi.T @ (Phi @ w_curr - y))
    w_next = w_curr + beta * (w_curr - w_old) - alpha * grad
    w_old = w_curr
    w_curr = w_next

w = w_curr
print('w =', w)

w = [ 0.18576016 -0.50140829 -0.73549508  0.24226256  0.11642292 -0.21034367
  0.12506324  0.13502261 -0.1395893   0.04124651 -0.45622021 -0.9758558
 -1.20808334 -0.35936482 -0.40182197 -0.42911394 -0.5957843  -0.60070047
 -0.46532017 -0.55327697 -0.53361229]


#### Prediction Error & Output
Given a single $x_i \in \mathbb R^d$, we predict $w^T\phi(x_i) = y_i \in \mathbb R$. Or equivalently, given multiple data points in form of a matrix $X$, we predict $\Phi w = y \in \mathbb R^n$, where $\Phi$ is the featurization of $X$. We use the Root Mean Squared Error for computing the prediction error on the training set. 

In [4]:
RMSE = mean_squared_error(y, Phi @ w) ** 0.5
print('RMSE =', RMSE)
np.savetxt("./results.csv", w, fmt="%.12f")

RMSE = 1.9476429547908933
